<a href="https://colab.research.google.com/github/atrabaja/MachineLearning_FinMark/blob/main/MS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [382]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [383]:
import os
data_path = '/content/drive/MyDrive'

In [384]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [385]:
# Define file paths
customers_file_path = '/content/drive/My Drive/customers_data.csv'
products_file_path = '/content/drive/My Drive/products_data.csv'
transactions_file_path = '/content/drive/My Drive/transactions_data.csv'

In [386]:
# Load datasets
customers_data = pd.read_csv(customers_file_path)
products_data = pd.read_csv(products_file_path)
transactions_data = pd.read_csv(transactions_file_path)

# Display the first few rows of each dataset
print(customers_data.head())
print(products_data.head())
print(transactions_data.head())

   Company_ID          Company_Name  Company_Profit  \
0         1.0  Tech  Enterprises  1         80701.0   
1         2.0   Global  Partners  2         80511.0   
2         3.0  Quantum Associates 3        110664.0   
3         4.0       Prime Network 4             NaN   
4         5.0    Elite  Ventures  5         69427.0   

                                             Address  
0             EDSA, Barangay 606, Pasig, Philippines  
1  Commonwealth Ave, Barangay 789, Taguig, Philip...  
2       Roxas Blvd, Barangay 505, Pasig, Philippines  
3  Alabang-Zapote Rd, Barangay 202, Taguig, Phili...  
4    Ayala Avenue, Barangay 101, Makati, Philippines  
   Product_ID            Product_Name Product_Price
0         1.0      FinPredictor Suite      ?140,000
1         2.0  MarketMinder Analytics      ?168,000
2         3.0    TrendWise Forecaster      ?100,800
3         4.0  CustomerScope Insights      ?123,200
4         5.0     SalesSync Optimizer       ?84,000
   Unnamed: 0  Transaction_

In [387]:
# Preprocessing Customers Data
# Identify rows with missing Company_ID
missing_ids = customers_data[customers_data['Company_ID'].isnull()]

# Map existing Company_IDs to rows with missing IDs based on matching columns
customers_data['Company_ID'] = customers_data['Company_ID'].fillna(
    customers_data.merge(
        customers_data[['Company_ID', 'Company_Name', 'Address', 'Company_Profit']].dropna(subset=['Company_ID']),
        on=['Company_Name', 'Address', 'Company_Profit'],
        how='left'
    )['Company_ID_y']
)

In [388]:
# Find the maximum existing Company_ID
max_id = customers_data['Company_ID'].max()

# Assign unique IDs to remaining missing rows
customers_data.loc[customers_data['Company_ID'].isnull(), 'Company_ID'] = range(
    int(max_id) + 1, int(max_id) + 1 + customers_data['Company_ID'].isnull().sum()
)

# Convert Company_ID to integer type
customers_data['Company_ID'] = customers_data['Company_ID'].astype(int)

In [389]:
# Check for duplicates based on Company_Name, Address, and Company_Profit
duplicates = customers_data[customers_data.duplicated(subset=['Company_Name', 'Address', 'Company_Profit'], keep=False)]
print(duplicates)

# Drop duplicates if necessary
customers_data = customers_data.drop_duplicates(subset=['Company_Name', 'Address', 'Company_Profit'], keep='first')

Empty DataFrame
Columns: [Company_ID, Company_Name, Company_Profit, Address]
Index: []


In [390]:
# Verify there are no missing values or duplicates
print(customers_data.isnull().sum())
print(customers_data.duplicated().sum())

# Display dataset summary
print(customers_data.info())
print(customers_data.head())

Company_ID         0
Company_Name       0
Company_Profit    12
Address            0
dtype: int64
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company_ID      100 non-null    int64  
 1   Company_Name    100 non-null    object 
 2   Company_Profit  88 non-null     float64
 3   Address         100 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.3+ KB
None
   Company_ID          Company_Name  Company_Profit  \
0           1  Tech  Enterprises  1         80701.0   
1           2   Global  Partners  2         80511.0   
2           3  Quantum Associates 3        110664.0   
3           4       Prime Network 4             NaN   
4           5    Elite  Ventures  5         69427.0   

                                             Address  
0             EDSA, Barangay 606, Pasig, Philippines  
1  Commonwealth Ave,

In [391]:
# Remove numbers from the Company_Name column
customers_data['Company_Name'] = customers_data['Company_Name'].str.replace(r'\d+', '', regex=True).str.strip()

# Verify the changes
print(customers_data['Company_Name'].head())

0     Tech  Enterprises
1      Global  Partners
2    Quantum Associates
3         Prime Network
4       Elite  Ventures
Name: Company_Name, dtype: object


In [392]:
# Fill missing Company_Profit based on Address group median
customers_data['Company_Profit'] = customers_data.groupby('Address')['Company_Profit'].transform(
    lambda x: x.fillna(x.median())
)

In [393]:
# Verify no missing values remain
print(customers_data.isnull().sum())

# Display updated dataset
print(customers_data.head())

Company_ID         0
Company_Name       0
Company_Profit    12
Address            0
dtype: int64
   Company_ID        Company_Name  Company_Profit  \
0           1   Tech  Enterprises         80701.0   
1           2    Global  Partners         80511.0   
2           3  Quantum Associates        110664.0   
3           4       Prime Network             NaN   
4           5     Elite  Ventures         69427.0   

                                             Address  
0             EDSA, Barangay 606, Pasig, Philippines  
1  Commonwealth Ave, Barangay 789, Taguig, Philip...  
2       Roxas Blvd, Barangay 505, Pasig, Philippines  
3  Alabang-Zapote Rd, Barangay 202, Taguig, Phili...  
4    Ayala Avenue, Barangay 101, Makati, Philippines  


In [394]:
# Cleaned customer data
customers_data.to_csv('/content/drive/MyDrive/cleaned_customers_data.csv', index=False)

In [395]:
# Preprocessing Transactions Data
# Check basic info
print(transactions_data.info())
print(transactions_data.head())

# Check for missing values
print(transactions_data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        9000 non-null   float64
 1   Transaction_ID    9000 non-null   float64
 2   Company_ID        9000 non-null   float64
 3   Product_ID        9000 non-null   float64
 4   Quantity          9000 non-null   float64
 5   Transaction_Date  10000 non-null  object 
 6   Product_Price     9000 non-null   float64
 7   Total_Cost        9000 non-null   float64
dtypes: float64(7), object(1)
memory usage: 625.1+ KB
None
   Unnamed: 0  Transaction_ID  Company_ID  Product_ID  Quantity  \
0         0.0             1.0        88.0         6.0       NaN   
1         1.0             2.0        29.0        19.0      16.0   
2         2.0             NaN        28.0        18.0       6.0   
3         3.0             4.0        85.0        12.0      12.0   
4         4.0             5.0

In [396]:
# Dropped unnamed column
transactions_data = transactions_data.drop(columns=['Unnamed: 0'])

In [397]:
# Handle Missing Values - Quality
transactions_data['Quantity'] = transactions_data['Quantity'].fillna(transactions_data['Quantity'].median())

In [398]:
# Handle Missing Values - Product Price
transactions_data['Product_Price'] = transactions_data['Product_Price'].fillna(transactions_data['Product_Price'].median())

In [400]:
# Display unique formats in Transaction_Date
print("Unique formats in Transaction_Date before conversion:\n", transactions_data['Transaction_Date'].unique())

# Define a function to parse different date formats
def parse_date(date_str):
    # Attempt to parse several known formats
    for fmt in ('%Y-%m-%d', '%d-%m-%Y', '%m-%d-%Y', '%Y/%m/%d', '%d/%m/%Y', '%m/%d/%Y', '%B %d, %Y', '%d %B %Y'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except (ValueError, TypeError):
            continue  # Try the next format if this one fails
    return pd.NaT  # Return NaT if no formats worked

# Apply the parsing function
transactions_data['Transaction_Date'] = transactions_data['Transaction_Date'].apply(parse_date)

# Check for NaT values after parsing
nat_count = transactions_data['Transaction_Date'].isna().sum()
print("NaT values in Transaction_Date column after conversion:", nat_count)

# Inspect problematic entries if there are any NaT values
if nat_count > 0:
    print("Remaining problematic date entries:\n", transactions_data[transactions_data['Transaction_Date'].isna()]['Transaction_Date'])

# Confirm the data type of Transaction_Date after all conversions
print("Data type of Transaction_Date after final conversion:", transactions_data['Transaction_Date'].dtypes)

Unique formats in Transaction_Date before conversion:
 <DatetimeArray>
['2024-03-26 00:00:00', '2024-07-09 00:00:00', '2024-04-13 00:00:00',
 '2023-06-09 00:00:00', '2021-06-07 00:00:00', '2021-07-12 00:00:00',
 '2023-11-01 00:00:00', '2023-10-31 00:00:00', '2024-02-25 00:00:00',
 '2021-03-29 00:00:00',
 ...
 '2022-11-30 00:00:00', '2024-03-16 00:00:00', '2020-06-11 00:00:00',
 '2020-01-12 00:00:00', '2022-07-13 00:00:00', '2024-11-08 00:00:00',
 '2024-12-04 00:00:00', '2023-11-04 00:00:00', '2020-03-12 00:00:00',
 '2024-12-09 00:00:00']
Length: 1490, dtype: datetime64[ns]
NaT values in Transaction_Date column after conversion: 0
Data type of Transaction_Date after final conversion: datetime64[ns]


In [401]:
# Final check of the dataset
print(transactions_data.info())
print(transactions_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_ID    9000 non-null   float64       
 1   Company_ID        9000 non-null   float64       
 2   Product_ID        9000 non-null   float64       
 3   Quantity          10000 non-null  float64       
 4   Transaction_Date  10000 non-null  datetime64[ns]
 5   Product_Price     10000 non-null  float64       
 6   Total_Cost        9000 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 547.0 KB
None
   Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
0             1.0        88.0         6.0      11.0       2024-03-26   
1             2.0        29.0        19.0      16.0       2024-07-09   
2             NaN        28.0        18.0       6.0       2024-04-13   
3             4.0        85.0        12.0      12.0  

In [402]:
# Verify Missing Values
print(transactions_data.isnull().sum())

Transaction_ID      1000
Company_ID          1000
Product_ID          1000
Quantity               0
Transaction_Date       0
Product_Price          0
Total_Cost          1000
dtype: int64


In [403]:
# Identify rows with missing Transaction_ID
missing_transaction_ids = transactions_data[transactions_data['Transaction_ID'].isnull()]
print(missing_transaction_ids)

      Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
2                NaN        28.0        18.0       6.0       2024-04-13   
17               NaN        57.0        10.0       6.0       2022-07-10   
21               NaN        25.0        18.0       2.0       2021-03-21   
25               NaN        93.0         9.0      11.0       2021-09-21   
55               NaN         NaN        11.0      16.0       2023-03-08   
...              ...         ...         ...       ...              ...   
9981             NaN         NaN         5.0      11.0       2022-03-29   
9984             NaN        93.0        19.0      11.0       2021-03-25   
9995             NaN         NaN        10.0      11.0       2022-06-05   
9997             NaN        90.0         1.0      15.0       2022-07-19   
9999             NaN        77.0        16.0      14.0       2023-10-29   

      Product_Price  Total_Cost  
2     126095.547778    940800.0  
17    131297.783516    627200.0

In [404]:
# Assign unique identifiers
# Find the maximum existing Transaction_ID
max_transaction_id = transactions_data['Transaction_ID'].max()

# Assign unique IDs to missing Transaction_IDs
transactions_data.loc[transactions_data['Transaction_ID'].isnull(), 'Transaction_ID'] = range(
    int(max_transaction_id) + 1,
    int(max_transaction_id) + 1 + transactions_data['Transaction_ID'].isnull().sum()
)

# Convert Transaction_ID to integer type (if necessary)
transactions_data['Transaction_ID'] = transactions_data['Transaction_ID'].astype(int)

# Verify updates
print(transactions_data['Transaction_ID'].isnull().sum())

0


In [405]:
# Identify rows with missing Company ID
missing_company_id = transactions_data[transactions_data['Company_ID'].isnull()]
print("Rows with missing Company_ID:\n", missing_company_id)

Rows with missing Company_ID:
       Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
6                  7         NaN         NaN      14.0       2023-11-01   
11                12         NaN         1.0      17.0       2022-04-05   
12                13         NaN        12.0      15.0       2023-02-27   
26                27         NaN        14.0      18.0       2023-03-01   
36                37         NaN         8.0      17.0       2023-03-16   
...              ...         ...         ...       ...              ...   
9962            9963         NaN        13.0       3.0       2023-03-14   
9965            9966         NaN         NaN       2.0       2022-11-13   
9972            9973         NaN         NaN      11.0       2021-04-20   
9981           10995         NaN         5.0      11.0       2022-03-29   
9995           10997         NaN        10.0      11.0       2022-06-05   

      Product_Price  Total_Cost  
6     133548.749710   1960000.0  


In [408]:
# Infer Company ID with other columns
# Create a mapping of Product_ID to Company_ID from rows where Company_ID is not null
product_to_company_map = transactions_data.dropna(subset=['Company_ID']).groupby('Product_ID')['Company_ID'].first()

# Use the mapping to fill missing Company_ID values
transactions_data['Company_ID'] = transactions_data.apply(
    lambda row: product_to_company_map[row['Product_ID']] if pd.isnull(row['Company_ID']) and row['Product_ID'] in product_to_company_map else row['Company_ID'],
    axis=1
)

In [409]:
# Recheck missing values
print(transactions_data['Company_ID'].isnull().sum())

93


In [410]:
# Investigate remaining missing rows
remaining_missing = transactions_data[transactions_data['Company_ID'].isnull()]
print("Remaining rows with missing Company_ID:\n", remaining_missing)

Remaining rows with missing Company_ID:
       Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
6                  7         NaN         NaN      14.0       2023-11-01   
41                42         NaN         NaN      18.0       2021-07-01   
183              184         NaN         NaN      13.0       2021-04-20   
292            10028         NaN         NaN       9.0       2021-04-03   
299              300         NaN         NaN      15.0       2023-08-14   
...              ...         ...         ...       ...              ...   
9769            9770         NaN         NaN       9.0       2022-01-08   
9773            9774         NaN         NaN      13.0       2023-04-07   
9907            9908         NaN         NaN      11.0       2022-04-22   
9965            9966         NaN         NaN       2.0       2022-11-13   
9972            9973         NaN         NaN      11.0       2021-04-20   

      Product_Price  Total_Cost  
6     133548.749710   19

In [411]:
# Check if rows with the same Transaction_Date have consistent Company_ID values.
# Create a mapping from Transaction_Date to Company_ID
date_to_company_map = transactions_data.dropna(subset=['Company_ID']).groupby('Transaction_Date')['Company_ID'].first()

# Use this mapping to fill missing Company_ID values
transactions_data['Company_ID'] = transactions_data.apply(
    lambda row: date_to_company_map.get(row['Transaction_Date']) if pd.isnull(row['Company_ID']) and row['Transaction_Date'] in date_to_company_map else row['Company_ID'],
    axis=1
)

In [413]:
# Recheck
print(transactions_data['Company_ID'].isnull().sum())

# Inspect remaining rows
print(transactions_data[transactions_data['Company_ID'].isnull()])

0
Empty DataFrame
Columns: [Transaction_ID, Company_ID, Product_ID, Quantity, Transaction_Date, Product_Price, Total_Cost]
Index: []


In [414]:
# Recalculate Total_Cost for missing values
transactions_data['Total_Cost'] = transactions_data.apply(
    lambda row: row['Quantity'] * row['Product_Price'] if pd.isnull(row['Total_Cost']) else row['Total_Cost'],
    axis=1
)
# Recheck
print(transactions_data.isnull().sum())

Transaction_ID         0
Company_ID             0
Product_ID          1000
Quantity               0
Transaction_Date       0
Product_Price          0
Total_Cost             0
dtype: int64


In [416]:
# Identify Rows with Missing Product_ID
missing_product_id = transactions_data[transactions_data['Product_ID'].isnull()]
print("Rows with missing Product_ID:\n", missing_product_id)

Rows with missing Product_ID:
       Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
6                  7        56.0         NaN      14.0       2023-11-01   
41                42        79.0         NaN      18.0       2021-07-01   
49                50        59.0         NaN       3.0       2023-01-13   
52                53        40.0         NaN      17.0       2021-10-24   
83                84        65.0         NaN      18.0       2023-04-22   
...              ...         ...         ...       ...              ...   
9961            9962         1.0         NaN       9.0       2022-05-07   
9963            9964        34.0         NaN      15.0       2023-06-20   
9965            9966        55.0         NaN       2.0       2022-11-13   
9972            9973        67.0         NaN      11.0       2021-04-20   
9998            9999        33.0         NaN      19.0       2021-04-15   

      Product_Price  Total_Cost  
6     133548.749710   1960000.0  


In [417]:
# Create a mapping of Product_Price to Product_ID
price_to_product_map = transactions_data.dropna(subset=['Product_ID']).groupby('Product_Price')['Product_ID'].first()

# Fill missing Product_ID based on Product_Price
transactions_data['Product_ID'] = transactions_data.apply(
    lambda row: price_to_product_map[row['Product_Price']] if pd.isnull(row['Product_ID']) and row['Product_Price'] in price_to_product_map else row['Product_ID'],
    axis=1
)

# Recheck
print("Remaining missing Product_ID values:", transactions_data['Product_ID'].isnull().sum())

Remaining missing Product_ID values: 895


In [418]:
# Infer using other columns
date_to_product_map = transactions_data.dropna(subset=['Product_ID']).groupby(['Transaction_Date', 'Company_ID'])['Product_ID'].first()

# Fill missing Product_ID based on Transaction_Date and Company_ID
transactions_data['Product_ID'] = transactions_data.apply(
    lambda row: date_to_product_map.get((row['Transaction_Date'], row['Company_ID']))
    if pd.isnull(row['Product_ID']) else row['Product_ID'],
    axis=1
)

In [419]:
# Recheck
print("Remaining missing Product_ID values:", transactions_data['Product_ID'].isnull().sum())

Remaining missing Product_ID values: 761


In [420]:
# Use Quantity and Total_Cost to infer Product_ID
quantity_cost_to_product_map = transactions_data.dropna(subset=['Product_ID']).groupby(['Quantity', 'Total_Cost'])['Product_ID'].first()

# Fill missing Product_ID based on Quantity and Total_Cost
transactions_data['Product_ID'] = transactions_data.apply(
    lambda row: quantity_cost_to_product_map.get((row['Quantity'], row['Total_Cost']))
    if pd.isnull(row['Product_ID']) and not pd.isnull(row['Quantity']) and not pd.isnull(row['Total_Cost'])
    else row['Product_ID'],
    axis=1
)

# Recheck
print("Remaining missing Product_ID values:", transactions_data['Product_ID'].isnull().sum())

Remaining missing Product_ID values: 96


In [422]:
# Infer Product_ID Using Additional Features
date_company_to_product_map = transactions_data.dropna(subset=['Product_ID']).groupby(['Transaction_Date', 'Company_ID'])['Product_ID'].first()

# Fill missing Product_ID based on Transaction_Date and Company_ID
transactions_data['Product_ID'] = transactions_data.apply(
    lambda row: date_company_to_product_map.get((row['Transaction_Date'], row['Company_ID']))
    if pd.isnull(row['Product_ID']) else row['Product_ID'],
    axis=1
)

# Recheck missing values
print("Remaining missing Product_ID values:", transactions_data['Product_ID'].isnull().sum())

Remaining missing Product_ID values: 94


In [423]:
# Verify
print(transactions_data.isnull().sum())

Transaction_ID       0
Company_ID           0
Product_ID          94
Quantity             0
Transaction_Date     0
Product_Price        0
Total_Cost           0
dtype: int64


In [424]:
# Fallback Strategy: Assign Placeholder IDs
max_product_id = transactions_data['Product_ID'].max()
transactions_data.loc[transactions_data['Product_ID'].isnull(), 'Product_ID'] = range(
    int(max_product_id) + 1,
    int(max_product_id) + 1 + transactions_data['Product_ID'].isnull().sum()
)
transactions_data['Product_ID'] = transactions_data['Product_ID'].astype(int)

In [425]:
# Verify
print(transactions_data.isnull().sum())

Transaction_ID      0
Company_ID          0
Product_ID          0
Quantity            0
Transaction_Date    0
Product_Price       0
Total_Cost          0
dtype: int64


In [426]:
# Save the Cleaned Dataset
transactions_data.to_csv('/content/drive/MyDrive/cleaned_transactions_data.csv', index=False)

In [427]:
# Preprocessing Products Data
# Basic information about the dataset
print(products_data.info())
print(products_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Product_ID     18 non-null     float64
 1   Product_Name   20 non-null     object 
 2   Product_Price  20 non-null     object 
dtypes: float64(1), object(2)
memory usage: 612.0+ bytes
None
   Product_ID            Product_Name Product_Price
0         1.0      FinPredictor Suite      ?140,000
1         2.0  MarketMinder Analytics      ?168,000
2         3.0    TrendWise Forecaster      ?100,800
3         4.0  CustomerScope Insights      ?123,200
4         5.0     SalesSync Optimizer       ?84,000


In [429]:
# Handle Missing Product_ID
max_product_id = products_data['Product_ID'].max()
products_data.loc[products_data['Product_ID'].isnull(), 'Product_ID'] = range(
    int(max_product_id) + 1,
    int(max_product_id) + 1 + products_data['Product_ID'].isnull().sum()
)
products_data['Product_ID'] = products_data['Product_ID'].astype(int)

# Recheck
print(products_data['Product_ID'].isnull().sum())

0


In [430]:
# Clean Product_Price Column
# Remove non-numeric characters and convert to float
products_data['Product_Price'] = products_data['Product_Price'].replace('[^\d.]', '', regex=True).astype(float)

In [431]:
# Verify and Handle Missing or Incorrect Prices
# Check for missing prices
print("Missing Product Prices:", products_data['Product_Price'].isnull().sum())

Missing Product Prices: 0


In [432]:
# Check for duplicates
duplicates = products_data[products_data.duplicated(subset=['Product_ID', 'Product_Name', 'Product_Price'], keep=False)]
print("Duplicate rows:\n", duplicates)

Duplicate rows:
 Empty DataFrame
Columns: [Product_ID, Product_Name, Product_Price]
Index: []


In [433]:
# Data validation
unmatched_products = transactions_data[~transactions_data['Product_ID'].isin(products_data['Product_ID'])]
print("Unmatched Product IDs:\n", unmatched_products)

Unmatched Product IDs:
       Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
10                11         6.0          16       6.0       2024-12-01   
30                31        16.0          16      20.0       2023-12-30   
33                34        92.0           8      11.0       2024-02-27   
36                37        92.0           8      17.0       2023-03-16   
69                70        71.0          16      18.0       2023-04-21   
...              ...         ...         ...       ...              ...   
9936            9937        14.0           8      17.0       2021-03-27   
9946            9947         9.0           8      11.0       2022-10-23   
9978            9979        37.0           8      14.0       2022-05-14   
9985            9986        14.0           8      13.0       2023-10-24   
9999           10999        77.0          16      14.0       2023-10-29   

      Product_Price    Total_Cost  
10     98263.850401  5.320000e+05  
30 

In [438]:
# Identify Unique Unmatched Product_IDs:
unmatched_ids = unmatched_products['Product_ID'].unique()
print("Unmatched Product IDs:", unmatched_ids)

Unmatched Product IDs: [ 16   8  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114]


In [439]:
# Investigate
# Create placeholder data for unmatched Product_IDs
unmatched_ids = unmatched_products['Product_ID'].unique()

# Create a list to hold the placeholder rows
placeholder_rows = []

for prod_id in unmatched_ids:
    placeholder_rows.append({
        'Product_ID': prod_id,
        'Product_Name': f'Unknown_{prod_id}',
        'Product_Price': None  # Or assign a default price if necessary
    })

# Convert placeholder rows into a DataFrame
placeholder_df = pd.DataFrame(placeholder_rows)

# Concatenate the placeholder rows to the original products data
products_data = pd.concat([products_data, placeholder_df], ignore_index=True)

# Re-check the unmatched products
unmatched_products = transactions_data[~transactions_data['Product_ID'].isin(products_data['Product_ID'])]
print("Remaining unmatched Product IDs:", unmatched_products)

Remaining unmatched Product IDs: Empty DataFrame
Columns: [Transaction_ID, Company_ID, Product_ID, Quantity, Transaction_Date, Product_Price, Total_Cost]
Index: []


<ipython-input-439-6a801b932c8a>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  products_data = pd.concat([products_data, placeholder_df], ignore_index=True)


In [440]:
# Double check unmatched
print("Unmatched Products DataFrame:")
print(unmatched_products)

Unmatched Products DataFrame:
Empty DataFrame
Columns: [Transaction_ID, Company_ID, Product_ID, Quantity, Transaction_Date, Product_Price, Total_Cost]
Index: []


In [441]:
# Merge Products Data and Transaction Data on Product ID
merged_data = pd.merge(transactions_data, products_data, on='Product_ID', how='left')
print(merged_data.head())

   Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
0               1        88.0           6      11.0       2024-03-26   
1               2        29.0          19      16.0       2024-07-09   
2           10000        28.0          18       6.0       2024-04-13   
3               4        85.0          12      12.0       2023-06-09   
4               5        47.0           3       8.0       2021-06-07   

   Product_Price_x  Total_Cost            Product_Name  Product_Price_y  
0    194379.147964   1075200.0    RevenueVue Dashboard         179200.0  
1     97930.993380   1428000.0        EcoNomix Modeler          95200.0  
2    126095.547778    940800.0  DashSync Analytics Hub         134400.0  
3    131297.783516   1008000.0        BudgetMaster Pro          84000.0  
4     99575.609634    705600.0    TrendWise Forecaster         100800.0  


In [442]:
# Merge datasets on 'Product_ID'
merged_data = pd.merge(transactions_data, products_data, on='Product_ID', how='left')

# Drop the unwanted 'Product_Price_x' column from transactions_data
merged_data = merged_data.drop(columns=['Product_Price_x'])

# Rename 'Product_Price_y' to 'Product_Price'
merged_data = merged_data.rename(columns={'Product_Price_y': 'Product_Price'})

# Display the cleaned merged data
print(merged_data.head())

   Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
0               1        88.0           6      11.0       2024-03-26   
1               2        29.0          19      16.0       2024-07-09   
2           10000        28.0          18       6.0       2024-04-13   
3               4        85.0          12      12.0       2023-06-09   
4               5        47.0           3       8.0       2021-06-07   

   Total_Cost            Product_Name  Product_Price  
0   1075200.0    RevenueVue Dashboard       179200.0  
1   1428000.0        EcoNomix Modeler        95200.0  
2    940800.0  DashSync Analytics Hub       134400.0  
3   1008000.0        BudgetMaster Pro        84000.0  
4    705600.0    TrendWise Forecaster       100800.0  


In [443]:
# Cleanup Mismatched Product ID
# Check for unmatched product ids in merged_data
unmatched_products = merged_data[merged_data['Product_Price'].isnull()]
print("Unmatched Products DataFrame:")
print(unmatched_products)

Unmatched Products DataFrame:
      Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
10                11         6.0          16       6.0       2024-12-01   
30                31        16.0          16      20.0       2023-12-30   
33                34        92.0           8      11.0       2024-02-27   
36                37        92.0           8      17.0       2023-03-16   
69                70        71.0          16      18.0       2023-04-21   
...              ...         ...         ...       ...              ...   
9936            9937        14.0           8      17.0       2021-03-27   
9946            9947         9.0           8      11.0       2022-10-23   
9978            9979        37.0           8      14.0       2022-05-14   
9985            9986        14.0           8      13.0       2023-10-24   
9999           10999        77.0          16      14.0       2023-10-29   

        Total_Cost Product_Name  Product_Price  
10    5.320000e+05  